# STEP 00 — Filter excluded labels & resplit 70/30

In [ ]:
TRAIN_CSV = "train.csv"
TEST_CSV  = "test.csv"
LABELS_CSV = "labels.csv"

TEXT_COL = "text"
DEMAND_COL = "demand_id"
EXCLUDE_COL = "exclude"

TEST_SIZE = 0.30
SEED = 42


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv(TRAIN_CSV)
test  = pd.read_csv(TEST_CSV)
labels = pd.read_csv(LABELS_CSV)

train[DEMAND_COL] = train[DEMAND_COL].astype(str)
test[DEMAND_COL]  = test[DEMAND_COL].astype(str)
labels[DEMAND_COL] = labels[DEMAND_COL].astype(str)

df = pd.concat([train, test], ignore_index=True)

excluded = set(labels.loc[labels[EXCLUDE_COL] == 1, DEMAND_COL])
df = df[~df[DEMAND_COL].isin(excluded)].copy()

print("Rows after exclude:", len(df))
print("Labels after exclude:", df[DEMAND_COL].nunique())


In [ ]:
counts = df[DEMAND_COL].value_counts()
single = counts[counts == 1].index

df_single = df[df[DEMAND_COL].isin(single)]
df_multi  = df[~df[DEMAND_COL].isin(single)]

train_multi, test_multi = train_test_split(
    df_multi,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=df_multi[DEMAND_COL]
)

train_clean = pd.concat([train_multi, df_single], ignore_index=True)
test_clean  = test_multi.copy()

print("Train size:", len(train_clean))
print("Test size :", len(test_clean))
print("Train labels:", train_clean[DEMAND_COL].nunique())
print("Test labels :", test_clean[DEMAND_COL].nunique())


In [ ]:
train_clean.to_csv("train_clean.csv", index=False)
test_clean.to_csv("test_clean.csv", index=False)
print("Saved train_clean.csv and test_clean.csv")